# Reading and Writing data on cloud object storage 
Reading from and writing to cloud object storage (e.g. AWS S3, Google Cloud Storage, Azure Blob Storage) is a bit different than regular filesystems.   Here we access public read buckets and write to an S3-API-compatible [Open Storage Network](https://www.openstoragenetwork.org) bucket.  We will make much use of `fsspec`, which offers filesystem interfaces to S3 in Python.

In [ ]:
import fsspec
import pandas as pd
import os
import xarray as xr

List files on a public read bucket

In [ ]:
fs = fsspec.filesystem('s3', anon=True)

In [ ]:
fs.ls('anaconda-public-datasets')

Reading CSV from a public read bucket

In [ ]:
df = pd.read_csv(fs.open("s3://anaconda-public-datasets/iris/iris.csv"))
df

Write CSV to an S3 bucket

In [ ]:
username = os.environ['JUPYTERHUB_USER'].split('@')[0]
print(username)

In [ ]:
fs_s3 = fsspec.filesystem('s3', profile='osn-esip', 
                          client_kwargs={'endpoint_url':'https://ncsa.osn.xsede.org'},
                          skip_instance_cache=True)


In [ ]:
bucket = 'esip'

In [ ]:
fs_s3.ls(bucket)

In [ ]:
outfile = fs_s3.open(f"s3://{bucket}/{username}/testing/iris.csv", 
                      mode='wt')

with outfile as f:
    df.to_csv(f)

List files on restricted S3 bucket

In [ ]:
fs_s3.ls(f'{bucket}/{username}/testing/')

The rest of the examples will use xarray, which follows the NetCDF data model

Read NetCDF data from THREDDS OPeNDAP Service  

In [ ]:
ds = xr.open_dataset('http://geoport.usgs.esipfed.org/thredds/dodsC/silt/usgs/Projects/stellwagen/CF-1.6/BUZZ_BAY/2651-A.cdf')
ds

In [ ]:
ds_nc = xr.open_dataset('http://armstrong.meas.ncsu.edu:8080/thredds/dodsC/fmrc/useast_v2_ROMS_L1_s-coord/COAWST-ROMS_L1_Sigma-Coordinate_FMRC_best.ncd')

In [ ]:
ds

Visualation interlude: plot a time range of data with hvplot

In [ ]:
import hvplot.xarray

ds['T_20'].sel(time=slice('1982-10-01','1982-10-31')).hvplot(grid=True)

Write NetCDF data to s3 bucket

In [ ]:
local_file = '2651-A-v3.nc'
s3_url = f's3://{bucket}/{username}/testing/2651-A-v3.nc'

ds.to_netcdf(local_file, mode='w')

_ = fs_s3.upload(local_file, s3_url)

Read NetCDF data from s3 bucket

In [ ]:
xr.open_dataset(fs_s3.open(s3_url))